In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Model, Input
from tensorflow.keras import layers
from math import sqrt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from pandas import concat
from pandas import read_csv
from helper import series_to_supervised, stage_series_to_supervised

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [3]:
dataset = pd.read_csv('../../data/Merged-update_hourly.csv', index_col=0)
dataset.fillna(0, inplace=True)
data = dataset
data

,WS_S1,WS_S4,FLOW_S25A,GATE_S25A,HWS_S25A,TWS_S25A,FLOW_S25B,GATE_S25B,GATE_S25B2,HWS_S25B,TWS_S25B,PUMP_S25B,FLOW_S26,GATE_S26_1,GATE_S26_2,HWS_S26,TWS_S26,PUMP_S26,MEAN_RAIN
Time,,,,,,,,,,,,,,,,,,,
1/1/10 00:00,0.33,0.34,0.0,0.0,2.17,0.49,0.0,0.0,0.0,2.84,0.58,0.0,0.00,0.0,0.00,2.78,0.505,0.0,0.0
1/1/10 01:00,-0.29,-0.28,0.0,0.0,2.17,0.02,0.0,0.0,0.0,2.84,-0.07,0.0,0.00,0.0,0.00,2.78,-0.110,0.0,0.0
1/1/10 02:00,-0.81,-0.73,0.0,0.0,2.17,-0.49,0.0,0.0,0.0,2.84,-0.62,0.0,0.00,0.0,0.00,2.79,-0.650,0.0,0.0
1/1/10 03:00,-0.98,-0.85,0.0,0.0,2.15,-0.75,0.0,0.0,0.0,2.83,-0.82,0.0,0.00,0.0,0.00,2.79,-0.850,0.0,0.0
1/1/10 04:00,-0.69,-0.49,0.0,0.0,2.13,-0.55,0.0,0.0,0.0,2.84,-0.53,0.0,0.00,0.0,0.00,2.79,-0.550,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12/31/20 19:00,1.24,1.19,0.0,0.0,1.92,1.25,0.0,0.0,0.0,2.86,1.31,0.0,456.34,0.0,2.61,2.49,1.290,0.0,0.0
12/31/20 20:00,1.62,1.61,0.0,0.0,1.92,1.64,0.0,0.0,0.0,2.88,1.68,0.0,380.19,0.0,2.62,2.48,1.660,0.0,0.0
12/31/20 21:00,1.98,1.90,0.0,0.0,1.93,2.00,0.0,0.0,0.0,2.89,2.04,0.0,287.12,0.0,2.62,2.54,2.040,0.0,0.0


In [4]:
print(dataset.columns)

Index(['WS_S1', 'WS_S4', 'FLOW_S25A', 'GATE_S25A', 'HWS_S25A', 'TWS_S25A',
       'FLOW_S25B', 'GATE_S25B', 'GATE_S25B2', 'HWS_S25B', 'TWS_S25B',
       'PUMP_S25B', 'FLOW_S26', 'GATE_S26_1', 'GATE_S26_2', 'HWS_S26',
       'TWS_S26', 'PUMP_S26', 'MEAN_RAIN'],
      dtype='object')


In [5]:
# specify the number of lag hours
n_hours = 24*3
K = 24

### Target time series (4)
- 'WS_S1',  'TWS_S25A', 'TWS_S25B', 'TWS_S26',    
        
               
### Prior known covariates (9)
- 'WS_S4', 'GATE_S25A', 'GATE_S25B', 'GATE_S25B2', 'PUMP_S25B',  'GATE_S26_1', 'GATE_S26_2', 'PUMP_S26', 'MEAN_RAIN'


### Prior unknown covariates (6)
- 'FLOW_S25A', 'HWS_S25A', 'FLOW_S25B', 'HWS_S25B', 'FLOW_S26', 'HWS_S26'

#### Target time series: water stage (4)

In [6]:
# Pre-processing
stages = dataset[['WS_S1', 'TWS_S25A', 'TWS_S25B', 'TWS_S26']]
print("stages.shape:", stages.shape)


stages_supervised = series_to_supervised(stages, n_hours, K)
print("stages_supervised.shape:", stages_supervised.shape)

stages.shape: (96432, 4)
stages_supervised.shape: (96337, 384)


In [7]:
stages_supervised

,var1(t-72),var2(t-72),var3(t-72),var4(t-72),var1(t-71),var2(t-71),var3(t-71),var4(t-71),var1(t-70),var2(t-70),...,var3(t+21),var4(t+21),var1(t+22),var2(t+22),var3(t+22),var4(t+22),var1(t+23),var2(t+23),var3(t+23),var4(t+23)
Time,,,,,,,,,,,,,,,,,,,,,
1/4/10 00:00,0.33,0.49,0.58,0.505,-0.29,0.02,-0.07,-0.11,-0.81,-0.49,...,1.07,1.04,1.45,1.61,1.63,1.58,1.91,2.05,2.08,2.02
1/4/10 01:00,-0.29,0.02,-0.07,-0.110,-0.81,-0.49,-0.62,-0.65,-0.98,-0.75,...,1.63,1.58,1.91,2.05,2.08,2.02,2.14,2.32,2.33,2.26
1/4/10 02:00,-0.81,-0.49,-0.62,-0.650,-0.98,-0.75,-0.82,-0.85,-0.69,-0.55,...,2.08,2.02,2.14,2.32,2.33,2.26,2.07,2.28,2.29,2.23
1/4/10 03:00,-0.98,-0.75,-0.82,-0.850,-0.69,-0.55,-0.53,-0.55,0.03,0.12,...,2.33,2.26,2.07,2.28,2.29,2.23,1.55,1.78,1.78,1.72
1/4/10 04:00,-0.69,-0.55,-0.53,-0.550,0.03,0.12,0.19,0.15,0.86,0.97,...,2.29,2.23,1.55,1.78,1.78,1.72,0.85,1.08,1.05,0.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12/30/20 20:00,1.93,1.97,2.03,1.990,1.48,1.48,1.59,1.54,0.98,0.97,...,0.23,0.21,0.77,0.74,0.75,0.75,1.24,1.25,1.31,1.29
12/30/20 21:00,1.48,1.48,1.59,1.540,0.98,0.97,1.05,1.01,0.58,0.61,...,0.75,0.75,1.24,1.25,1.31,1.29,1.62,1.64,1.68,1.66
12/30/20 22:00,0.98,0.97,1.05,1.010,0.58,0.61,0.72,0.66,0.04,0.03,...,1.31,1.29,1.62,1.64,1.68,1.66,1.98,2.00,2.04,2.04


#### Prior unknown covariates (6)

In [8]:
prior_unknown = dataset[['FLOW_S25A', 'HWS_S25A', 'FLOW_S25B', 'HWS_S25B', 'FLOW_S26', 'HWS_S26' ]]
print("prior_unknown.shape:", prior_unknown.shape)

prior_unknown_supervised = series_to_supervised(prior_unknown, n_hours, K)
print("prior_unknown_supervised.shape:", prior_unknown_supervised.shape)

prior_unknown.shape: (96432, 6)
prior_unknown_supervised.shape: (96337, 576)


In [9]:
# prior_unknown_supervised

#### Prior known covariates (9)

In [10]:
prior_known = dataset[['WS_S4', 'GATE_S25A', 'GATE_S25B', 'GATE_S25B2', 'PUMP_S25B', 'GATE_S26_1', 'GATE_S26_2', 'PUMP_S26', 'MEAN_RAIN']]
print("prior_known.shape:", prior_known.shape)

prior_known_supervised = series_to_supervised(prior_known, n_hours, K)
print("prior_known_supervised.shape:", prior_known_supervised.shape)

prior_known.shape: (96432, 9)
prior_known_supervised.shape: (96337, 864)


In [11]:
# prior_known_supervised

### N_out, N_in, K

In [12]:
past_cov = dataset[['FLOW_S25A', 'HWS_S25A', 'FLOW_S25B', 'HWS_S25B', 'FLOW_S26', 'HWS_S26',\
                   'WS_S4', 'GATE_S25A', 'GATE_S25B', 'GATE_S25B2', 'PUMP_S25B', \
                    'GATE_S26_1', 'GATE_S26_2', 'PUMP_S26', 'MEAN_RAIN']]

past_cov_supervised = series_to_supervised(past_cov, n_hours, K)
print("past_cov_supervised.shape:", past_cov_supervised.shape)

past_cov_supervised.shape: (96337, 1440)


In [13]:
# past_cov_supervised  # 6 + 9 = 15 features

In [14]:
past_ws_supervised = stages_supervised.iloc[:, :n_hours*stages.shape[1]]
past_ws_supervised.reset_index(drop=True, inplace=True)
past_ws_supervised.columns = ['past_ws_supervised_' + i for i in list(past_ws_supervised.columns)]

In [15]:
# past_ws_supervised  # 4

In [16]:
columns = []
for i in range(n_hours):
    columns = columns + past_cov_supervised.columns[i*past_cov.shape[1]:(i+1)*past_cov.shape[1]].tolist()
    columns = columns + past_ws_supervised.columns[i*stages.shape[1]:(i+1)*stages.shape[1]].tolist()
# columns

In [17]:
past_cov_supervised.reset_index(drop=True, inplace=True)
past_ws_supervised.reset_index(drop=True, inplace=True)

past_cov_ws_supervised = pd.concat([past_cov_supervised, past_ws_supervised], axis=1)
past_cov_ws_supervised = past_cov_ws_supervised[columns]


In [18]:
# past_cov_ws_supervised

In [19]:
shift_prior_known_supervised = prior_known_supervised.iloc[:, K*prior_known.shape[1]:]  # shift 2 days for prior known covariates
shift_prior_known_supervised.reset_index(drop=True, inplace=True)
shift_prior_known_supervised.columns = ['shift_prior_known_supervised_' + i for i in list(shift_prior_known_supervised.columns)]
shift_prior_known_supervised

,shift_prior_known_supervised_var1(t-48),shift_prior_known_supervised_var2(t-48),shift_prior_known_supervised_var3(t-48),shift_prior_known_supervised_var4(t-48),shift_prior_known_supervised_var5(t-48),shift_prior_known_supervised_var6(t-48),shift_prior_known_supervised_var7(t-48),shift_prior_known_supervised_var8(t-48),shift_prior_known_supervised_var9(t-48),shift_prior_known_supervised_var1(t-47),...,shift_prior_known_supervised_var9(t+22),shift_prior_known_supervised_var1(t+23),shift_prior_known_supervised_var2(t+23),shift_prior_known_supervised_var3(t+23),shift_prior_known_supervised_var4(t+23),shift_prior_known_supervised_var5(t+23),shift_prior_known_supervised_var6(t+23),shift_prior_known_supervised_var7(t+23),shift_prior_known_supervised_var8(t+23),shift_prior_known_supervised_var9(t+23)
0,0.60,0.0,0.0,0.00,0.0,0.0,2.61,0.0,0.0,-0.07,...,0.0,1.99,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
1,-0.07,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,-0.68,...,0.0,2.17,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
2,-0.68,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,-1.02,...,0.0,2.06,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
3,-1.02,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,-0.97,...,0.0,1.50,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
4,-0.97,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,-0.50,...,0.0,0.86,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96332,1.91,0.0,0.0,2.57,0.0,0.0,2.57,0.0,0.0,1.67,...,0.0,1.19,0.0,0.0,0.0,0.0,0.0,2.61,0.0,0.0
96333,1.67,0.0,0.0,2.57,0.0,0.0,2.57,0.0,0.0,1.16,...,0.0,1.61,0.0,0.0,0.0,0.0,0.0,2.62,0.0,0.0
96334,1.16,0.0,0.0,2.57,0.0,0.0,2.57,0.0,0.0,0.68,...,0.0,1.90,0.0,0.0,0.0,0.0,0.0,2.62,0.0,0.0
96335,0.68,0.0,0.0,2.57,0.0,0.0,2.56,0.0,0.0,0.24,...,0.0,2.03,0.0,0.0,0.0,0.0,0.0,2.62,0.0,0.0


In [20]:
shift_prior_known_past_cov_ws_supervised = pd.concat([past_cov_ws_supervised, shift_prior_known_supervised], 
                                                     axis=1)


In [21]:
# shift_prior_known_past_cov_ws_supervised

In [22]:
future_ws_supervised = stages_supervised.iloc[:, n_hours*stages.shape[1]:]
future_ws_supervised.reset_index(drop=True, inplace=True)
future_ws_supervised

,var1(t),var2(t),var3(t),var4(t),var1(t+1),var2(t+1),var3(t+1),var4(t+1),var1(t+2),var2(t+2),...,var3(t+21),var4(t+21),var1(t+22),var2(t+22),var3(t+22),var4(t+22),var1(t+23),var2(t+23),var3(t+23),var4(t+23)
0,2.07,2.27,2.26,2.22,1.57,1.80,1.80,1.72,0.81,1.04,...,1.07,1.04,1.45,1.61,1.63,1.58,1.91,2.05,2.08,2.02
1,1.57,1.80,1.80,1.72,0.81,1.04,1.03,0.96,0.27,0.53,...,1.63,1.58,1.91,2.05,2.08,2.02,2.14,2.32,2.33,2.26
2,0.81,1.04,1.03,0.96,0.27,0.53,0.50,0.42,-0.29,-0.02,...,2.08,2.02,2.14,2.32,2.33,2.26,2.07,2.28,2.29,2.23
3,0.27,0.53,0.50,0.42,-0.29,-0.02,-0.06,-0.14,-0.68,-0.41,...,2.33,2.26,2.07,2.28,2.29,2.23,1.55,1.78,1.78,1.72
4,-0.29,-0.02,-0.06,-0.14,-0.68,-0.41,-0.49,-0.57,-0.64,-0.46,...,2.29,2.23,1.55,1.78,1.78,1.72,0.85,1.08,1.05,0.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96332,1.93,1.95,2.00,2.00,2.19,2.21,2.25,2.25,2.11,2.12,...,0.23,0.21,0.77,0.74,0.75,0.75,1.24,1.25,1.31,1.29
96333,2.19,2.21,2.25,2.25,2.11,2.12,2.22,2.19,1.65,1.69,...,0.75,0.75,1.24,1.25,1.31,1.29,1.62,1.64,1.68,1.66
96334,2.11,2.12,2.22,2.19,1.65,1.69,1.76,1.72,1.24,1.24,...,1.31,1.29,1.62,1.64,1.68,1.66,1.98,2.00,2.04,2.04
96335,1.65,1.69,1.76,1.72,1.24,1.24,1.37,1.32,0.76,0.73,...,1.68,1.66,1.98,2.00,2.04,2.04,2.13,2.13,2.21,2.18


### Concatenation

In [23]:
all_data = concat([shift_prior_known_past_cov_ws_supervised, future_ws_supervised], axis=1)
# print("all_data", all_data)
print("all_data.shape:", all_data.shape)

all_data.shape: (96337, 2112)


### Train & Test set

In [24]:
all_data = all_data.values
n_train_hours = int(len(all_data)*0.8)
print("n_train_hours:", n_train_hours)


train = all_data[:n_train_hours, :]
test = all_data[n_train_hours:, :]

n_train_hours: 77069


In [25]:
n_obs = n_hours * (prior_known.shape[1]+data.shape[1])
n_obs

2016

### Normalization

In [26]:
# split into input and outputs
all_features = prior_known.shape[1] + data.shape[1]
n_obs = n_hours * all_features
train_X, train_y = train[:, :n_obs], train[:, n_obs:]
test_X, test_y = test[:, :n_obs], test[:, n_obs:]
print("train_X.shape, train_y.shape, test_X.shape, test_y.shape", train_X.shape, train_y.shape, test_X.shape, test_y.shape)


# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
train_X = scaler.fit_transform(train_X)
train_y = scaler.fit_transform(train_y)
test_X = scaler.fit_transform(test_X)
test_y = scaler.fit_transform(test_y)

train_X.shape, train_y.shape, test_X.shape, test_y.shape (77069, 2016) (77069, 96) (19268, 2016) (19268, 96)


In [27]:
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], n_hours, all_features))
test_X = test_X.reshape((test_X.shape[0], n_hours, all_features))
print("train_X.shape, train_y.shape, test_X.shape, test_y.shape: \n", train_X.shape, train_y.shape, test_X.shape, test_y.shape)

train_X.shape, train_y.shape, test_X.shape, test_y.shape: 
 (77069, 72, 28) (77069, 96) (19268, 72, 28) (19268, 96)


### Model

In [29]:
model_input = Input(shape=(train_X.shape[1], train_X.shape[2]))
x = layers.SimpleRNN(128, 
                     activation='relu', 
                     kernel_regularizer=keras.regularizers.L1L2(l1=1e-5, l2=1e-5),
                     recurrent_regularizer=keras.regularizers.L1L2(l1=0, l2=1e-5),
                     return_sequences=True)(model_input)
x = layers.SimpleRNN(64, 
                     activation='relu', 
                     kernel_regularizer=keras.regularizers.L1L2(l1=1e-5, l2=1e-5),
                     recurrent_regularizer=keras.regularizers.L1L2(l1=1e-5, l2=1e-5),
                     return_sequences=True)(x)
x = layers.SimpleRNN(32, 
                     activation='relu', 
                     kernel_regularizer=keras.regularizers.L1L2(l1=1e-5, l2=1e-5),
                     recurrent_regularizer=keras.regularizers.L1L2(l1=1e-5, l2=1e-5),
                     return_sequences=True)(x)
x = layers.SimpleRNN(16, 
                     activation='relu', 
                     kernel_regularizer=keras.regularizers.L1L2(l1=1e-5, l2=1e-5),
                     recurrent_regularizer=keras.regularizers.L1L2(l1=1e-5, l2=1e-5),
                     return_sequences=True)(x)

x = layers.Flatten()(x)
# x = layers.Dense(256, activation='relu', kernel_regularizer=keras.regularizers.L1L2(l1=1e-5, l2=1e-4))(x)
# x = layers.Dense(128, activation='relu', kernel_regularizer=keras.regularizers.L1L2(l1=1e-4, l2=1e-4))(x)
# x = layers.Dense(64, activation='relu', kernel_regularizer=keras.regularizers.L1L2(l1=1e-5, l2=1e-4))(x)

output = layers.Dense(train_y.shape[1])(x)

model_rnn = Model(model_input, output)
model_rnn.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 72, 28)]          0         
_________________________________________________________________
simple_rnn_4 (SimpleRNN)     (None, 72, 128)           20096     
_________________________________________________________________
simple_rnn_5 (SimpleRNN)     (None, 72, 64)            12352     
_________________________________________________________________
simple_rnn_6 (SimpleRNN)     (None, 72, 32)            3104      
_________________________________________________________________
simple_rnn_7 (SimpleRNN)     (None, 72, 16)            784       
_________________________________________________________________
flatten_1 (Flatten)          (None, 1152)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 96)                1106

In [ ]:
lr = 0.0001
EPOCHS = 8000

# initial_learning_rate * decay_rate ^ (step / decay_steps)
lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=5e-4, 
                                                          decay_steps=10000,
                                                          decay_rate=0.99)

model_rnn.compile(
    optimizer=Adam(learning_rate=lr, decay=lr/EPOCHS), 
#               optimizer=Adam(learning_rate=lr_schedule), 
#               optimizer='adam',
              loss='mse',
              metrics=['mae'])

es = EarlyStopping(monitor='val_loss', mode='min', verbose=2, patience=1000)
mc = ModelCheckpoint('saved_model/rnn_shift_anyday_24h_4layer.h5', monitor='val_mae', mode='min', verbose=2, save_best_only=True)


history = model_rnn.fit(train_X, train_y,
                    batch_size=512,
                    epochs=EPOCHS,
                    validation_data=(test_X, test_y),
                    verbose=2,
                    shuffle=True,
                       callbacks=[es, mc])

plt.rcParams["figure.figsize"] = (8, 6)
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlabel('Epoch', fontsize=16)
plt.ylabel('Loss', fontsize=16)
plt.legend(fontsize=14)
plt.title("Training loss vs Testing loss", fontsize=18)
# plt.savefig('graph/rnn_loss.png', dpi=300)
plt.show()

Epoch 1/8000


2023-01-07 17:10:37.716135: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10



Epoch 00001: val_mae improved from inf to 0.10664, saving model to saved_model/rnn_shift_anyday_24h_4layer.h5
151/151 - 21s - loss: 0.0630 - mae: 0.1463 - val_loss: 0.0417 - val_mae: 0.1066
Epoch 2/8000

Epoch 00002: val_mae improved from 0.10664 to 0.07348, saving model to saved_model/rnn_shift_anyday_24h_4layer.h5
151/151 - 21s - loss: 0.0289 - mae: 0.0562 - val_loss: 0.0319 - val_mae: 0.0735
Epoch 3/8000

Epoch 00003: val_mae improved from 0.07348 to 0.06551, saving model to saved_model/rnn_shift_anyday_24h_4layer.h5
151/151 - 21s - loss: 0.0257 - mae: 0.0430 - val_loss: 0.0290 - val_mae: 0.0655
Epoch 4/8000

Epoch 00004: val_mae improved from 0.06551 to 0.06069, saving model to saved_model/rnn_shift_anyday_24h_4layer.h5
151/151 - 21s - loss: 0.0240 - mae: 0.0378 - val_loss: 0.0270 - val_mae: 0.0607
Epoch 5/8000

Epoch 00005: val_mae improved from 0.06069 to 0.05638, saving model to saved_model/rnn_shift_anyday_24h_4layer.h5
151/151 - 21s - loss: 0.0227 - mae: 0.0344 - val_loss: 0.

Epoch 46/8000

Epoch 00046: val_mae did not improve from 0.03340
151/151 - 21s - loss: 0.0034 - mae: 0.0174 - val_loss: 0.0045 - val_mae: 0.0341
Epoch 47/8000

Epoch 00047: val_mae did not improve from 0.03340
151/151 - 21s - loss: 0.0033 - mae: 0.0174 - val_loss: 0.0044 - val_mae: 0.0339
Epoch 48/8000

Epoch 00048: val_mae improved from 0.03340 to 0.03334, saving model to saved_model/rnn_shift_anyday_24h_4layer.h5
151/151 - 20s - loss: 0.0032 - mae: 0.0174 - val_loss: 0.0043 - val_mae: 0.0333
Epoch 49/8000

Epoch 00049: val_mae did not improve from 0.03334
151/151 - 20s - loss: 0.0031 - mae: 0.0174 - val_loss: 0.0042 - val_mae: 0.0334
Epoch 50/8000

Epoch 00050: val_mae improved from 0.03334 to 0.03299, saving model to saved_model/rnn_shift_anyday_24h_4layer.h5
151/151 - 20s - loss: 0.0030 - mae: 0.0173 - val_loss: 0.0041 - val_mae: 0.0330
Epoch 51/8000

Epoch 00051: val_mae did not improve from 0.03299
151/151 - 20s - loss: 0.0029 - mae: 0.0173 - val_loss: 0.0040 - val_mae: 0.0333
Ep

Epoch 97/8000

Epoch 00097: val_mae did not improve from 0.03153
151/151 - 21s - loss: 0.0017 - mae: 0.0161 - val_loss: 0.0027 - val_mae: 0.0320
Epoch 98/8000

Epoch 00098: val_mae did not improve from 0.03153
151/151 - 20s - loss: 0.0016 - mae: 0.0161 - val_loss: 0.0027 - val_mae: 0.0317
Epoch 99/8000

Epoch 00099: val_mae did not improve from 0.03153
151/151 - 20s - loss: 0.0016 - mae: 0.0160 - val_loss: 0.0027 - val_mae: 0.0317
Epoch 100/8000

Epoch 00100: val_mae improved from 0.03153 to 0.03150, saving model to saved_model/rnn_shift_anyday_24h_4layer.h5
151/151 - 21s - loss: 0.0016 - mae: 0.0160 - val_loss: 0.0026 - val_mae: 0.0315
Epoch 101/8000

Epoch 00101: val_mae did not improve from 0.03150
151/151 - 20s - loss: 0.0016 - mae: 0.0160 - val_loss: 0.0026 - val_mae: 0.0316
Epoch 102/8000

Epoch 00102: val_mae did not improve from 0.03150
151/151 - 21s - loss: 0.0016 - mae: 0.0160 - val_loss: 0.0026 - val_mae: 0.0317
Epoch 103/8000

Epoch 00103: val_mae did not improve from 0.031

...

Epoch 6555/8000

Epoch 06555: val_mae did not improve from 0.02081
151/151 - 20s - loss: 4.1574e-04 - mae: 0.0098 - val_loss: 0.0010 - val_mae: 0.0211
Epoch 6556/8000

Epoch 06556: val_mae did not improve from 0.02081
151/151 - 20s - loss: 4.1572e-04 - mae: 0.0098 - val_loss: 0.0010 - val_mae: 0.0212
Epoch 6557/8000

Epoch 06557: val_mae did not improve from 0.02081
151/151 - 20s - loss: 4.1590e-04 - mae: 0.0098 - val_loss: 9.9549e-04 - val_mae: 0.0210
Epoch 6558/8000

Epoch 06558: val_mae did not improve from 0.02081
151/151 - 22s - loss: 4.1563e-04 - mae: 0.0098 - val_loss: 9.9179e-04 - val_mae: 0.0209
Epoch 6559/8000

Epoch 06559: val_mae did not improve from 0.02081
151/151 - 20s - loss: 4.1571e-04 - mae: 0.0098 - val_loss: 0.0010 - val_mae: 0.0212
Epoch 6560/8000

Epoch 06560: val_mae did not improve from 0.02081
151/151 - 20s - loss: 4.1570e-04 - mae: 0.0098 - val_loss: 0.0010 - val_mae: 0.0214
Epoch 6561/8000

Epoch 06561: val_mae did not improve from 0.02081
151/151 - 20s - los

Epoch 6609/8000

Epoch 06609: val_mae did not improve from 0.02081
151/151 - 20s - loss: 4.1519e-04 - mae: 0.0098 - val_loss: 0.0010 - val_mae: 0.0212
Epoch 6610/8000

Epoch 06610: val_mae did not improve from 0.02081
151/151 - 20s - loss: 4.1536e-04 - mae: 0.0098 - val_loss: 0.0010 - val_mae: 0.0212
Epoch 6611/8000

Epoch 06611: val_mae did not improve from 0.02081
151/151 - 20s - loss: 4.1530e-04 - mae: 0.0098 - val_loss: 0.0010 - val_mae: 0.0211
Epoch 6612/8000

Epoch 06612: val_mae did not improve from 0.02081
151/151 - 20s - loss: 4.1570e-04 - mae: 0.0098 - val_loss: 0.0010 - val_mae: 0.0215
Epoch 6613/8000

Epoch 06613: val_mae did not improve from 0.02081
151/151 - 20s - loss: 4.1522e-04 - mae: 0.0098 - val_loss: 0.0010 - val_mae: 0.0212
Epoch 6614/8000

Epoch 06614: val_mae did not improve from 0.02081
151/151 - 20s - loss: 4.1568e-04 - mae: 0.0098 - val_loss: 0.0010 - val_mae: 0.0212
Epoch 6615/8000

Epoch 06615: val_mae did not improve from 0.02081
151/151 - 20s - loss: 4.154

Epoch 6663/8000

Epoch 06663: val_mae did not improve from 0.02081
151/151 - 20s - loss: 4.1490e-04 - mae: 0.0098 - val_loss: 0.0010 - val_mae: 0.0215
Epoch 6664/8000

Epoch 06664: val_mae did not improve from 0.02081
151/151 - 20s - loss: 4.1484e-04 - mae: 0.0098 - val_loss: 9.7719e-04 - val_mae: 0.0209
Epoch 6665/8000

Epoch 06665: val_mae did not improve from 0.02081
151/151 - 22s - loss: 4.1512e-04 - mae: 0.0098 - val_loss: 0.0010 - val_mae: 0.0211
Epoch 6666/8000

Epoch 06666: val_mae did not improve from 0.02081
151/151 - 21s - loss: 4.1487e-04 - mae: 0.0098 - val_loss: 0.0010 - val_mae: 0.0213
Epoch 6667/8000

Epoch 06667: val_mae did not improve from 0.02081
151/151 - 20s - loss: 4.1495e-04 - mae: 0.0098 - val_loss: 9.9829e-04 - val_mae: 0.0211
Epoch 6668/8000

Epoch 06668: val_mae did not improve from 0.02081
151/151 - 20s - loss: 4.1475e-04 - mae: 0.0098 - val_loss: 0.0010 - val_mae: 0.0213
Epoch 6669/8000

Epoch 06669: val_mae did not improve from 0.02081
151/151 - 20s - los

Epoch 6717/8000

Epoch 06717: val_mae did not improve from 0.02081
151/151 - 20s - loss: 4.1455e-04 - mae: 0.0098 - val_loss: 0.0010 - val_mae: 0.0213
Epoch 6718/8000

Epoch 06718: val_mae did not improve from 0.02081
151/151 - 20s - loss: 4.1455e-04 - mae: 0.0098 - val_loss: 0.0010 - val_mae: 0.0212
Epoch 6719/8000

Epoch 06719: val_mae did not improve from 0.02081
151/151 - 20s - loss: 4.1456e-04 - mae: 0.0098 - val_loss: 9.9006e-04 - val_mae: 0.0211
Epoch 6720/8000

Epoch 06720: val_mae did not improve from 0.02081
151/151 - 20s - loss: 4.1454e-04 - mae: 0.0098 - val_loss: 9.8865e-04 - val_mae: 0.0210
Epoch 6721/8000

Epoch 06721: val_mae did not improve from 0.02081
151/151 - 20s - loss: 4.1455e-04 - mae: 0.0098 - val_loss: 9.8680e-04 - val_mae: 0.0210
Epoch 6722/8000

Epoch 06722: val_mae did not improve from 0.02081
151/151 - 20s - loss: 4.1443e-04 - mae: 0.0098 - val_loss: 0.0010 - val_mae: 0.0212
Epoch 6723/8000

Epoch 06723: val_mae did not improve from 0.02081
151/151 - 20s -

Epoch 6771/8000

Epoch 06771: val_mae did not improve from 0.02081
151/151 - 20s - loss: 4.1436e-04 - mae: 0.0098 - val_loss: 0.0010 - val_mae: 0.0213
Epoch 6772/8000

Epoch 06772: val_mae did not improve from 0.02081
151/151 - 19s - loss: 4.1440e-04 - mae: 0.0098 - val_loss: 0.0010 - val_mae: 0.0211
Epoch 6773/8000

Epoch 06773: val_mae did not improve from 0.02081
151/151 - 20s - loss: 4.1404e-04 - mae: 0.0098 - val_loss: 0.0010 - val_mae: 0.0212
Epoch 6774/8000

Epoch 06774: val_mae did not improve from 0.02081
151/151 - 20s - loss: 4.1434e-04 - mae: 0.0098 - val_loss: 0.0010 - val_mae: 0.0213
Epoch 6775/8000

Epoch 06775: val_mae did not improve from 0.02081
151/151 - 20s - loss: 4.1423e-04 - mae: 0.0098 - val_loss: 0.0010 - val_mae: 0.0211
Epoch 6776/8000

Epoch 06776: val_mae did not improve from 0.02081
151/151 - 21s - loss: 4.1412e-04 - mae: 0.0098 - val_loss: 9.8589e-04 - val_mae: 0.0210
Epoch 6777/8000

Epoch 06777: val_mae did not improve from 0.02081
151/151 - 21s - loss: 4

### Performance

In [30]:
from tensorflow.keras.models import load_model

saved_model = load_model('saved_model/rnn_shift_anyday_24h_4layer.h5')

In [31]:
yhat = saved_model.predict(test_X)
inv_yhat = scaler.inverse_transform(yhat)
inv_y = scaler.inverse_transform(test_y)

inv_yhat = pd.DataFrame(inv_yhat)
inv_y = pd.DataFrame(inv_y)
print("inv_y.shape, inv_yhat.shape", inv_y.shape, inv_yhat.shape)

error = inv_yhat - inv_y
print(error.shape)

2023-02-04 18:52:06.821203: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10


inv_y.shape, inv_yhat.shape (19268, 96) (19268, 96)
(19268, 96)


In [32]:
print('MAE = {}'.format(float("{:.3f}".format(mae(inv_yhat.iloc[:, :], inv_y.iloc[:, :])))))
print('RMSE = {}'.format(float("{:.3f}".format(sqrt(mse(inv_yhat.iloc[:, :], inv_y.iloc[:, :]))))))

MAE = 0.11
RMSE = 0.151
